In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
from time import sleep
import urllib3
import json
from selenium import webdriver
import random

In [2]:
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    parser = html.fromstring(resp)
    urllib3.disable_warnings()

    return parser

In [3]:
parser_link_base = parser('https://fapesp.br/chamadas/')

links_anos_brutos = parser_link_base.xpath('//div[@class="box"][3]//a//@href')

base = 'https://fapesp.br/'
link_anos = []

for l in links_anos_brutos:
    link = base + l
    link_anos.append(link)


/home/duban/anaconda3/envs/scraping-clacso/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'fapesp.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [4]:
fapesp = pd.DataFrame()

for link in link_anos:

    parser_ano = parser(link)
    #print(link)

    #######################

    #Titulo

    titulo = parser_ano.xpath('//div[@class="page-body"]//text()')
    
    titulos = []
    for t in titulo:
        if (t.startswith('Chamada') or t.startswith('Programa ')) and '/' not in t:
            titulos.append(t)

    ################################

    #Links
    
    links = []

    for i in range(1, len(titulos)+1):

        url = parser_ano.xpath('//div[@class="page-body"]/ul//li[' +str(i)+ ']//a/@href')
        try:
            links.append(url[0])
        except:
            links.append('')
        

    #link = parser_ano.xpath('//div[@class="page-body"]/ul//li/p/strong[1]//a/@href')

    ###################

    ano = parser_ano.xpath('//div[@class="content row"]//h2/span/text()')[0].strip().split(' ')[-1]

    fapesp_ano = pd.DataFrame()
    fapesp_ano['Titulo'] = titulos
    fapesp_ano['Link'] = links
    fapesp_ano['Año'] = ano

    fapesp = pd.concat([fapesp, fapesp_ano])
fapesp = fapesp.reset_index(drop=True)

In [7]:
pdfs_proyectos = []
descripcion_proyectos = []


for link in fapesp['Link']:

    parser_proyecto = parser(fapesp['Link'][0])

    descripcion_bruto = parser_proyecto.xpath('//div[@class="content row"]//text()')

    descripcion = ''

    for d in descripcion_bruto:
        d = d.strip()
        descripcion = descripcion + d + ' '

    descripcion_proyectos.append(descripcion)

    pdfs_brutos = parser_proyecto.xpath('//div[@class="content row"]//a/@href')

    pdfs = ''

    for p in pdfs_brutos:
        p = p.strip()
        if p.endswith('.pdf'):
            pdfs = pdfs + p + ' '

    pdfs_proyectos.append(pdfs)

fapesp['PDFs'] = pdfs_proyectos
fapesp['Descripcion'] = descripcion_proyectos

In [11]:
fapesp.to_excel('brasil_fapesp.xlsx')